In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

### dependencies

In [ ]:
pg = pd.read_csv('/home/lindb/teakettle/postmoran/popgentx.txt',sep='\t')
#pg.replace(np.nan,"NA")
pg.head()

In [ ]:
loci = sorted(pg.columns.tolist()[1:])
Chloro = loci[:3]
nuc = loci[3:]
loci

In [ ]:
nuc

In [ ]:
Chloro

### missing data including null alleles

In [ ]:
for locus in loci:
    print locus
    print pg[locus].tolist().count(np.nan)/len(pg[locus].tolist())

In [ ]:
txs = uni(pg['pop'])[:3]
txs

In [ ]:
for tx in txs:
    print '\t',tx
    df = pg[pg['pop']==tx].copy()
    for locus in loci:
        print locus
        print ' ',df[locus].tolist().count(np.nan)/len(df[locus].tolist())
    print ''

### distinct chlorotypes

In [ ]:
pg.head()

In [ ]:
chloros = []
chloroplots = {}
for samp in pg.index:
    if pg.loc[samp,:].tolist()[-3:].count(np.nan) == 0:
        chloro = "-".join(pg.loc[samp,:].tolist()[-3:])
        if not chloro in chloros:
            chloros.append(chloro)
            chloroplots[chloro] = []
            if not pg.loc[samp,'pop'] in chloroplots[chloro]:
                chloroplots[chloro].append(pg.loc[samp,'pop'])
len(chloros)

In [ ]:
len(chloroplots)

In [ ]:
for c in chloroplots:
    if len(chloroplots[c]) > 1:
        print chloroplots[c]

In [ ]:
len(chloroplots)/nrow(pg)

In [ ]:
# there are no shared chlorotypes among treatments

# Genetic diversity parameters

In [ ]:
# create dict for dataframe
d = OrderedDict()
for tx in txs: # tx is the line, keys of d[tx] will be genetic diversity parameters
    d[tx] = OrderedDict()

### samp sizes (N) and alleles per tx (A_t) 

In [ ]:
pg.head()

In [ ]:
for tx in txs:
    print tx
    df = pg[pg['pop']==tx].copy()
    if not 'N' in d[tx]:
        d[tx]['N'] = nrow(df)
    print nrow(df)
    tot = 0
    for locus in loci:
        ls = [l for l in df[locus].tolist() if type(l)==str]
        ls = uni([l for x in ls for l in x.split(":")])
        tot += len(ls)
    if not 'A_t' in d[tx]:
        d[tx]['A_t'] = tot
    print 'tot=',tot    

In [ ]:
['new' in x for x in pg.index].count(False)

In [ ]:
for tx in txs:
    print tx
    df = pg[pg['pop']==tx].copy()
    news = ['new' in x for x in df.index].count(True)
    ads  = nrow(df) - news
    assert news + ads == nrow(df)
    print 'new',news
    print 'adults',ads

### mean alleles per loc (A) , + std dev

In [ ]:
from scipy import stats

In [ ]:
for tx in txs:
    df = pg[pg['pop']==tx].copy()
    tot = {}
    for locus in loci:
        ls = [l for l in df[locus].tolist() if type(l)==str]
        ls = uni([l for x in ls for l in x.split(":")])
        tot[locus] = len(ls)
    s = np.std(tot.values())
    
    d[tx]['A'] = "%.2f (%.2f)" % (np.mean(tot.values()),
                                  s)

### effective number of alleles per locus (A_e)
take harmonic mean across loci

In [ ]:
Chloro

In [ ]:
nuc

In [ ]:
# calc A_e for each locus, take harmonic mean
Aes = {} # keep track of 
for tx in txs:
    df = pg[pg['pop']==tx].copy()
    Aes[tx] = {}
    for locus in loci:
        gts = [l for l in df[locus].tolist() if type(l)==str] # gives each genotype, len = samps with genotypes
        if locus in nuc:
            denom = 2*len(gts)
            allAs = [l for x in gts for l in x.split(":")] # all alleles
        if locus in Chloro:
            denom = len(gts)
            allAs = [x.split(":")[0] for x in gts] # only first allele since homozygous
        uAs   = uni(allAs) # unique alleles
        sums = 0
        for a in uAs:
            freq = allAs.count(a)/denom # nallele / 2N or nallel / N
            sums += (freq**2)
        sums = 1/sums
        Aes[tx][locus] = sums
    mn = stats.hmean(Aes[tx].values())
    se = np.std(Aes[tx].values())
    d[tx]['A_e'] = "%.2f (%.2f)" % (stats.hmean(Aes[tx].values()),
                                    se)

In [ ]:
d

### observed and expected heterozygosity - nuclear markers only

In [ ]:
hobs   = {} # keep track of 
hexps = {}
for tx in txs:
    df = pg[pg['pop']==tx].copy()
    hobs[tx]   = {}
    hexps[tx] = {}
    for locus in nuc:
        gts = [l for l in df[locus].tolist() if type(l)==str] # gives each genotype, len = samps with genotypes
        twoN = 2*len(gts) #2N
        hobs[tx][locus] = 0
        for g in gts: # calc Hobs
            one,two = g.split(":")
            if one != two:
                hobs[tx][locus] += 1
        hobs[tx][locus] = hobs[tx][locus]/len(gts)
        
        allAs = [l for x in gts for l in x.split(":")] # all alleles
        uAs   = uni(allAs)
        exphom = [] # list of expected homozygosities
        for a in uAs: # get exphom for each allele
            freq = allAs.count(a)/twoN
            exphom.append(freq**2)
        hexps[tx][locus] = 1-sum(exphom)
        
    mhexp   = np.mean(hexps[tx].values())
    mhexpsd = np.std(hexps[tx].values())
    mhobs   = np.mean(hobs[tx].values())
    mhobssd = np.std(hobs[tx].values())
    
    d[tx]['Ho'] = "%.2f (%.2f)" % (mhobs,mhobssd)
    d[tx]['He'] = "%.2f (%.2f)" % (mhexp,mhexpsd)
        

In [ ]:
d

### private alleles (A_p)

In [ ]:
# get a list of unique alleles per locus
Ls = {}
for locus in loci:
    gts = [g for g in pg[locus].tolist() if type(g)==str]
    allAs = [l for x in gts for l in x.split(":")]
    Ls[locus] = uni(allAs)

In [ ]:
# get a list of unique alleles per treatment
pls = {}
for tx in txs:
    pls[tx] = {}
    df = pg[pg['pop']==tx].copy()
    for locus in loci:
        gts   = [g for g in df[locus].tolist() if type(g)==str]
        allAs = [l for x in gts for l in x.split(":")]
        pls[tx][locus]   = uni(allAs)

In [ ]:
### private alleles per tx
prv = {}
# bc
print 'bc'
prv['bc'] = {}
for locus in loci:
    alleles = pls['bc'][locus]
    alleles = [l for l in alleles if not l in pls['uc'][locus]+pls['un'][locus]]
    prv['bc'][locus] = alleles
    print locus,len(prv['bc'][locus])
# uc
print 'uc'
prv['uc'] = {}
for locus in loci:
    alleles = pls['uc'][locus]
    alleles = [l for l in alleles if not l in pls['bc'][locus]+pls['un'][locus]]
    prv['uc'][locus] = alleles
    print locus,len(prv['uc'][locus])
# un
print 'un'
prv['un'] = {}
for locus in loci:
    alleles = pls['un'][locus]
    alleles = [l for l in alleles if not l in pls['bc'][locus]+pls['uc'][locus]]
    prv['un'][locus] = alleles
    print locus,len(prv['un'][locus])

In [ ]:
for tx in txs:
    d[tx]['A_p'] = sum([len(x) for x in prv[tx].values()])

In [ ]:
d

### overall mean for each category

In [ ]:
d['Overall Mean'] = OrderedDict()
for cat in d[tx]:
    vals = [d['bc'][cat],d['uc'][cat],d['un'][cat]]
    vals = [str(val).split()[0] for val in vals]
    print vals
    val = np.mean([float(v) for v in vals])
    d['Overall Mean'][cat] = "%.2f" % val

In [ ]:
d

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/sumstats'
if not op.exists(DIR):
    mkdir(DIR)
cd(DIR)

In [ ]:
filE = op.join(DIR,'sumstats.txt')
key0 = d.keys()[0]
header = 'Treatment'+'\t'+'\t'.join([x for x in d[key0].keys()]) + '\n'
with open(filE,'w') as o:
    o.write("%s" % header)
    for tx in d:
        text = tx + '\t'+'\t'.join([str(x) for x in d[tx].values()]) + '\n'
        o.write("%s" % text)
o.close()
sumstats = pd.read_csv(filE,header=0,sep='\t')
sumstats.head()

# hierfstat

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/sumstats/hierf/RDS'
if not op.exists(DIR):
    mkdir(DIR)

#### R magic voodoo

In [ ]:
def setup_r(): 
    os.environ['R_HOME'] = '/home/lindb/g/R3/lib64/R/' 
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'])

In [ ]:
setup_r()
import readline
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri as pd2ri
pd2ri.activate()
r = robjects.r

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython
%matplotlib inline

In [ ]:
%%R
sessionInfo()

In [ ]:
pg.head()

In [ ]:
%%R
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/sumstats/hierf/RDS')
library(adegenet)
library(hierfstat)
library(snow)
pg   <- read.csv('/home/lindb/teakettle/postmoran/popgentx.txt',sep='\t')
pg <- pg[which(pg[,'pop'] != 'us'),]
loci <- colnames(pg)[-1]
pg[,'plot'] <- substr(rownames(pg),1,3)

pops <- pg[,'pop']
plot <- pg[,'plot']
pg <- pg[,c('pop','plot',loci)]
tfk <- rep('tfk',nrow(pg))
strat <- data.frame(cbind(tfk,pops,plot))
hierar = ~TFK/Treatment/Plot

### get hierfstat df
pg2 <- pg[,c(-1,-2)]
gentfk <- df2genind(pg2,sep=":",pop=pops)
pgout <- genind2df(gentfk)
whiches <- which(rownames(pg2) %in% rownames(pgout))
strat <- strat[whiches,]
pops <- pgout[,'pop']
pgout <- pgout[,-1]
gentfk <- df2genind(pgout,sep=":",pop=pops,strata=strat,hierarchy=~tfk/pops/plot)
hierdf <- genind2hierfstat(gentfk)
head(hierdf)
hierdf[,'pop'] <- as.character(hierdf[,'pop'])
hierdf[,'pop'] <- replace(hierdf[,'pop'], hierdf[,'pop']=='bc', 1)
hierdf[,'pop'] <- replace(hierdf[,'pop'], hierdf[,'pop']=='uc', 2)
hierdf[,'pop'] <- replace(hierdf[,'pop'], hierdf[,'pop']=='un', 3)
hierdf[,'plot'] <- substr(rownames(pgout),3,3)

hierdf <- hierdf[,c('pop','plot',loci)]

### functions
get_varcomp = function(x) 
{
    library(hierfstat)
    loci = data.frame(x)
    res <- varcomp(cbind(levels, loci),diploid=T)$overall
}

finish_varcomp = function(m) 
{
    tot <- apply(m, 2, sum, na.rm = TRUE)
    nblevels <- length(tot)
    f <- matrix(rep(0, (nblevels - 1)^2), ncol = (nblevels - 1))
    for (i in 1:(nblevels - 1)) 
    {
        for (j in i:(nblevels - 1)) 
        {
            f[i, j] <- sum(tot[i:j])/sum(tot[i:nblevels])
        }
    }
    row.names(m) <- lnames
    print(names(tot))
    tf <- t(f)
    row.names(tf) <- fnames
    f <- t(tf)
    row.names(f) <- c("Total", fnames[-length(fnames)])
    return(list(loc = m, overall = tot, F = f))
}

### do the hierfstat
levels = data.frame(hierdf[,1:2])
Loci   = hierdf[,loci]
lnames = names(Loci)
fnames= c(names(levels),"Ind")
cl = makeSOCKcluster(30)
clusterExport(cl, "levels", envir=environment())
system.time(res <- matrix(parCapply(cl, Loci, get_varcomp), nrow=length(names(Loci)),byrow=T))
res = finish_varcomp(res)
saveRDS(res, "hierfstatRUN.RDS")
system.time(bs <- basic.stats(gentfk))
saveRDS(bs, "bs_hierfstatRUN.RDS")
stopCluster(cl)
print("Done!")

pg2gt <- pg2[which(rownames(pg2) %in% rownames(pgout)),]

#rds()

In [ ]:
def get_r_series(key):
    s = pd.Series(get_r(key))
    s.index = get_r("names(%s)" % key)
    return s

def get_r_df(key):
    df = pd.DataFrame(get_r(key))
    try:
        rname = get_r("rownames(%s)" % key)
        df.index = rname
    except:
        pass
    
    try:
        cname = get_r("colnames(%s)" % key)
        df.columns = cname
    except:
        pass
    
    return df

def get_r(key):
    return r(key)

In [ ]:
%%R
res = readRDS("/home/lindb/teakettle/postmoran/sumstats/hierf/RDS/hierfstatRUN.RDS")

In [ ]:
%%R
bs  = readRDS("/home/lindb/teakettle/postmoran/sumstats/hierf/RDS/bs_hierfstatRUN.RDS")

In [ ]:
resF = get_r_df("res$F")
resF

In [ ]:
bs = get_r_df("bs$perloc")
bs.head()

In [ ]:
overall = get_r_df("res$overall")
overall

In [ ]:
loc = get_r_df("res$loc")
loc.head()

In [ ]:
#multilocus FST
def compute_F_plotTotal(series):
    Hs = series[0]+series[1]
    Ht = sum(series)
    return Hs/Ht

In [ ]:
plot_total_fst = loc.apply(compute_F_plotTotal, axis=1)
plot_total_fst

In [ ]:
def compute_F_popTotal(series):
    return series[0]/sum(series)

In [ ]:
pop_total_fst = loc.apply(compute_F_popTotal, axis = 1)
pop_total_fst

In [ ]:
bs['Fst']

In [ ]:
def compute_F_plotPop(series):
    return series[1]/sum(series[1:])

In [ ]:
plot_pop_fst = loc.apply(compute_F_plotPop,axis=1)
plot_pop_fst

In [ ]:
dfs = [pop_total_fst,plot_total_fst,plot_pop_fst]
ds = ["pop_total","plot_total","plot_pop"]
for i,d in enumerate(dfs):
    d = pd.DataFrame(d)
    d.columns = [ds[i]]
    if i == 0:
        df = pd.DataFrame(d)
    else:
        df = pd.merge(df,d,left_index=True,right_index=True)
df.head()

In [ ]:
%%R
hierdf = readRDS('/home/lindb/teakettle/postmoran/sumstats/hierf/RDS/hierdf.RDS')

In [ ]:
hierdf = get_r('hierdf')
hierdf.replace(-2147483648,'NA',inplace=True)

In [ ]:
hierdf.head()

### pairwise FST

In [ ]:
%%R
# edited pairwise.WCfst from hierfstat
library(hierfstat)
library(adegenet)
pg2gt <- readRDS('/home/lindb/teakettle/postmoran/sumstats/hierf/RDS/pg2gt.RDS')
pg2gt[,'pop'] <- substr(rownames(pg2gt),1,2)
pg2gt[,'pop'] <- replace(pg2gt[,'pop'], pg2gt[,'pop']=='bc', 1)
pg2gt[,'pop'] <- replace(pg2gt[,'pop'], pg2gt[,'pop']=='uc', 2)
pg2gt[,'pop'] <- replace(pg2gt[,'pop'], pg2gt[,'pop']=='un', 3)
pg2gt <- pg2gt[,c('pop',colnames(pg2gt)[-5])]
pwFST <- function (dat, diploid = TRUE) #### edited from hierfstat - see commented line for change
{
    ### pairwise.WCfst
    dat <- dat[order(dat[, 1]), ]
    pops <- unique(dat[, 1])
    npop <- length(pops)
    fstmat <- matrix(nrow = npop, ncol = npop, dimnames = list(pops,
        pops))
    rownames(fstmat) <- colnames(fstmat) <- c('bc','uc','un')
    if (is.factor(dat[, 1])) {
        dat[, 1] <- as.numeric(dat[, 1])
        pops <- as.numeric(pops)
    }
    
    for (a in 2:npop) {
        for (b in 1:(a - 1)) {
            subdat <- dat[dat[, 1] == pops[a] | dat[, 1] == pops[b],
                ]
            popz <- subdat[,1]
            fstmat[a, b] <- fstmat[b, a] <- wc(df2genind(subdat[,-1],sep=":",pop=popz), diploid = diploid)$FST
#             fstmat[a, b] <- fstmat[b, a] <- wc(subdat, diploid = diploid)$FST
        }
    }
    fstmat
}
pw <- pwFST(pg2gt)
pw[-1,-3]

In [ ]:
%%R
.mean(pw[-1,-3])

### FST for each treatment, with plots nested in tx

In [ ]:
%%R
rm(list=ls())
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/sumstats/hierf/RDS')
library(adegenet)
library(hierfstat)
library(snow)
pg   <- read.csv('/home/lindb/teakettle/postmoran/popgentx.txt',sep='\t')
pg   <- pg[which(pg[,'pop'] != 'us'),]
loci <- colnames(pg)[-1]
nuc <- loci[1:4]
pg[,'plot'] <- substr(rownames(pg),3,3)
pg[,'pop']  <- replace(pg[,'pop'], pg[,'pop']=='bc', 1)
pg[,'pop']  <- replace(pg[,'pop'], pg[,'pop']=='uc', 2)
pg[,'pop']  <- replace(pg[,'pop'], pg[,'pop']=='un', 3)
pg <- pg[,c('pop','plot',nuc)]
head(pg)

In [ ]:
%%R
print(uni(pg$pop))
print(uni(pg$plot))

In [ ]:
%%R
bc <- pg[which(pg[,'pop']=='1'),-1]
uc <- pg[which(pg[,'pop']=='2'),-1]
un <- pg[which(pg[,'pop']=='3'),-1]
print(c(nrow(bc),nrow(uc),nrow(un)))

In [ ]:
%%R
print(c(uni(bc[,'plot']),
        uni(uc[,'plot']),
        uni(un[,'plot'])
       ))

In [ ]:
%%R
bcplots <- bc[,'plot']
bch <- df2genind(bc[,-1],sep=":",pop=bcplots)
# ucplots <- uc[which(uc[,'plot']!=1),'plot']
# uch <- df2genind(uc[which(uc[,'plot']!=1),-1],sep=":",pop=ucplots)
ucplots <- uc[,'plot']
uch <- df2genind(uc[,-1],sep=":",pop=ucplots)
unplots <- un[,'plot']
unh <- df2genind(un[,-1],sep=":",pop=unplots)

In [ ]:
%%R
c(table(bcplots),table(ucplots),table(unplots))

In [ ]:
%%R
c(table(genind2df(bch)[,1]),
  table(genind2df(uch)[,1]),
  table(genind2df(unh)[,1]))

In [ ]:
%%R
c(table(genind2df(bch)[,1]),
  table(genind2df(uch)[,1]),
  table(genind2df(unh)[,1]))

In [ ]:
%%R
bcwc <- wc(bch)
ucwc <- wc(uch)
unwc <- wc(unh)

In [ ]:
%%R
names(bcwc)

In [ ]:
%%R
plotFST <- data.frame('FST'=rep('NA',3),'FIS'=rep('NA',3))
plotFST[1,] <- c(bcwc$FST,bcwc$FIS)
plotFST[2,] <- c(ucwc$FST,ucwc$FIS)
plotFST[3,] <- c(unwc$FST,unwc$FIS)

In [ ]:
plotFST = get_r_df('plotFST')
for row in plotFST.index:
    for col in plotFST.columns:
        plotFST.loc[row,col] = round(float(plotFST.loc[row,col]),4)
plotFST.index = ['bc','uc','un']
plotFST

# Exclusion probabilities

In [ ]:
pg.head()

In [ ]:
%%R
# get parentage assignment
DIR = '/home/lindb/teakettle/data/gibbs/first/end_of_cyc_files'
f = readRDS(fname(DIR,'f.RDS'))
m = readRDS(fname(DIR,'m.RDS'))
DIR = '/home/lindb/teakettle/data/moran/dependencies/'
bframe1 = readRDS(fname(DIR,'bframe1.RDS'))
bframe2 = readRDS(fname(DIR,'bframe2.RDS'))
geno = read.csv('/home/lindb/teakettle/data/moran/moran_infile_final_jan_wconecounts.txt',sep='\t')
head(geno)

In [ ]:
%%R
# get samp names
adults <- bframe1[seq(1,nrow(bframe1),2),'samp']
seedlings <- bframe2[seq(1,nrow(bframe2),2),'samp']

In [ ]:
%%R
adults[26]

In [ ]:
# pull into python
moms = get_r_df('m')
dads = get_r_df('f')
adults = get_r_df('adults')
seedlings = get_r_df('seedlings')
len(adults),len(seedlings)

In [ ]:
# translate between row number and samp name
adict = {}
for row in adults.index:
    adict[row] = adults.loc[row,0]
sdict = {}
for row in seedlings.index:
    sdict[row] = seedlings.loc[row,0]
adict[600] = 'out-of-plot'

In [ ]:
adict[600]

In [ ]:
# assign samp names to moms and dads
moms.index = sdict.values()
dads.index = sdict.values()
momma = {}
dadda = {}
for row in moms.index.tolist():
    momname = adict[moms.loc[row,0]]
    moms.loc[row,0] = momname
    momma[row] = momname
    
    dadname = adict[dads.loc[row,0]]
    dads.loc[row,0] = dadname
    dadda[row] = dadname

In [ ]:
pg['mom'] = ''
pg['dad'] = ''
pg['OffID'] = pg.index.tolist()
pg.head()

In [ ]:
# fill in data
for row in pg.index.tolist():
    if row in seedlings[0].tolist():
        pg.loc[row,'mom'] = momma[row]
        pg.loc[row,'dad'] = dadda[row]
    else:
        pg.loc[row,'mom'] = '0'
        pg.loc[row,'dad'] = '0'

In [ ]:
pg.head()

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/moran_analysis'
if not op.exists(DIR):
    mkdir(DIR)
pg.to_csv(op.join(DIR,'assignments.txt'),sep="\t",index=True,header=True)
pg2 = pd.read_csv(op.join(DIR,'assignments.txt'),header=0,index_col=0,sep="\t")
pg2.head()

In [ ]:
# get a dataframe for adults
adultdf = pg[pg['mom']=='0'].copy()
adultdf.head()

In [ ]:
# get a dataframe for seedlings
seeddf = pg[pg['mom'] != '0'].copy()
seeddf.head()

In [ ]:
def freqs(df,All='all',loci=loci):
    assert All in ['nuc','all','chloro']
    if All=='nuc':
        loci = sorted(loci)[3:]
    elif All=='chloro':
        loci = sorted(loci)[:3]
    af = {}
    af['Locus'] = {}
    af['Allele'] = {}
    af['Freq'] = {}
    count = 0
    for locus in loci:
        gts = [g for g in df[locus].tolist() if type(g)==str]
        allAs = [l for x in gts for l in x.split(":")]
        for a in uni(allAs):
            af['Locus'][count] = locus
            af['Allele'][count] = a
            af['Freq'][count] = allAs.count(a)/len(allAs)
            count += 1
    af = pd.DataFrame(af)
    af = af.loc[:,['Locus','Allele','Freq']]
    return af
def get_excl(data):
    fp = 0
    sp = 0
    data.index = [str(x) for x in range(len(data.index))]
    ind = data.index.tolist()
    l = len(ind)
#     return (ind,l)
    for rowi,i in enumerate(ind):
        pi = data['Freq'][rowi]
        fp = fp + pi * (1 - pi)**2
        for rowj,j in enumerate(ind):
            if i != j:
                pj = data['Freq'][rowj]
                sp = sp + pi**2 * pj**2 * (4 - 3 * pi - 3 * pj)
    Pexcl = (fp - 0.5 * sp)
    PexclMax = 1 - (2 * l**3 + l**2 - 5 * l + 3)/l**4
    return (Pexcl,PexclMax)     
def exclusion_prob(f):
    loci = uni(f['Locus'])
    ret = {}
    count = 0
    ret['Locus'] = {}
    ret['Pexcl'] = {}
    ret['PexclMax'] = {}
    ret['Fraction'] = {}
    for locus in loci:
        data = f[f['Locus'] == locus].copy()
#         return data
        (Pexcl,PexclMax) = get_excl(pd.DataFrame(data))
#         return (Pexcl,PexclMax)
        ret['Locus'][count] = locus
        ret['Pexcl'][count] = Pexcl
        ret['PexclMax'][count] = PexclMax
        ret['Fraction'][count] = Pexcl/PexclMax
        count += 1
    ret = pd.DataFrame(ret)
    ret = ret.loc[:,['Locus','Pexcl','PexclMax','Fraction']]
    return pd.DataFrame(ret)
def multexcl(df):
    assert 'Pexcl' in df.columns.tolist()
    omin = [1-float(x) for x in df['Pexcl'].tolist()]
    prod = np.prod(omin)
    pmult = 1 - prod
    return pmult

In [ ]:
# afs = freqs(pg)
adultsfreqs = freqs(adultdf)
afreqsnuc   = freqs(adultdf,All='nuc')
afreqschl   = freqs(adultdf,All='chloro')

In [ ]:
# probability of exclusion for each locus
allexcl = exclusion_prob(adultsfreqs)
allexcl # chloro- and nuc-specific excl probabilities won't change from this below

In [ ]:
# multilocus exclusion probability
pmult = multexcl(allexcl)
pmult

In [ ]:
# look at nuclear exclusion probability
nucexcl = exclusion_prob(afreqsnuc)
nucmult = multexcl(nucexcl)
nucmult

In [ ]:
# look at chloro exclusion probability
chlexcl = exclusion_prob(afreqschl)
chlmult = multexcl(chlexcl)
chlmult

In [ ]:
[x for x in dir() if not str.startswith(x,'_')]

# prob of pedigree vs missing data

In [ ]:
f = '/home/lindb/teakettle/postmoran/moran_analysis/pars.pkl'
par = pickle.load(open(f))
f = '/home/lindb/teakettle/postmoran/moran_analysis/adults.pkl'
As = pickle.load(open(f))

In [ ]:
pg.head()

In [ ]:
pg.loc[samp,loci].tolist().count(np.nan)

In [ ]:
# calculate missing data
ms = {}
for samp in pg.index:
    ms[samp] = pg.loc[samp,loci].tolist().count(np.nan)

In [ ]:
missing = []
probs = []
for seed in par:
    df = par[seed].copy()
    df.loc[:,'mom'] = [As[int(x)] for x in df['mom'].tolist()]
    df.loc[:,'dad'] = [As[int(x)] for x in df['dad'].tolist()]
    df = df[df['mom'].isin(adults[0].tolist())].copy()
    df = df[df['dad'].isin(adults[0].tolist())].copy()
    if nrow(df) > 1:
        mom,dad,prob = df.loc[df.prob.idxmax(),:]
        m = sum([ms[mom],ms[dad],ms[seed]])/21
        missing.append(m)
        probs.append(prob)

In [ ]:
plt.scatter(missing,probs)
plt.xlabel('Missing data')
plt.ylabel('Probability')

In [ ]:
len(np.arange(5))

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(missing,probs)
xi = np.arange(0,len(missing))
A = np.array([xi,np.ones(len(missing))])
line = slope*xi+intercept
plt.plot(missing,probs,'o',missing,line)

In [ ]:
slope

In [ ]:
xs = [x*2 for x in range(20)]
ys = [y for y in range(20)]
plt.scatter(xs,ys)